In [17]:
import random
from PIL import Image, ImageOps
import os
import time
import numpy as np
import matplotlib.pyplot as plt
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchvision.models import resnet50
from torch.utils.data import DataLoader, Subset, random_split
from sklearn.model_selection import KFold
from tqdm import tqdm

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Seed for reproducibility
SEED = 309
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True


Using device: cuda


In [18]:

# Dataset paths
dataset_path = 'train_data'
output_path = 'processed_data'
classes = ['tomato', 'cherry', 'strawberry']
target_size = (128, 128)

# Ensure output directory exists
os.makedirs(output_path, exist_ok=True)
for cls in classes:
    os.makedirs(os.path.join(output_path, cls), exist_ok=True)

# Blurry image detection function
def is_blurry(image):
    gray_image = np.array(image.convert('L'))
    laplacian_var = cv2.Laplacian(gray_image, cv2.CV_64F).var()
    return laplacian_var > 2000  # Threshold for blurriness

count = 0
# Preprocess images
for fruit_class in classes:
    folder_path = os.path.join(dataset_path, fruit_class)
    output_folder_path = os.path.join(output_path, fruit_class)
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        image = Image.open(image_path)

        # Check for blur
        if is_blurry(image):
            print(f"Skipping blurry image: {image_path}")
            count += 1
            continue
        
        # Resize and save preprocessed image
        image = ImageOps.fit(image, target_size, Image.LANCZOS)
        processed_image_path = os.path.join(output_folder_path, image_file)
        image.save(processed_image_path)

print("Preprocessing completed. Processed images are saved in the 'processed_data' folder.")
print(f"Skipped {count} bad images.")


Skipping blurry image: train_data\tomato\tomato_0055.jpg
Skipping blurry image: train_data\tomato\tomato_0136.jpg
Skipping blurry image: train_data\tomato\tomato_0153.jpg
Skipping blurry image: train_data\tomato\tomato_0156.jpg
Skipping blurry image: train_data\tomato\tomato_0157.jpg
Skipping blurry image: train_data\tomato\tomato_0158.jpg
Skipping blurry image: train_data\tomato\tomato_0228.jpg
Skipping blurry image: train_data\tomato\tomato_0241.jpg
Skipping blurry image: train_data\tomato\tomato_0261.jpg
Skipping blurry image: train_data\tomato\tomato_0307.jpg
Skipping blurry image: train_data\tomato\tomato_0399.jpg
Skipping blurry image: train_data\tomato\tomato_0433.jpg
Skipping blurry image: train_data\tomato\tomato_0500.jpg
Skipping blurry image: train_data\tomato\tomato_0666.jpg
Skipping blurry image: train_data\tomato\tomato_0765.jpg
Skipping blurry image: train_data\tomato\tomato_0793.jpg
Skipping blurry image: train_data\tomato\tomato_0820.jpg
Skipping blurry image: train_da

In [19]:
# Define transformations
transform_train = transforms.Compose([
    transforms.Resize(target_size),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=(-30, 30)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

transform_test = transforms.Compose([
    transforms.Resize(target_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


In [20]:
# Define data directories
data_root = './train_data/'
preprocessed_data_root = './processed_data/'

# Load datasets with transformations
train_dataset = ImageFolder(root=data_root, transform=transform_train)
test_dataset = ImageFolder(root=preprocessed_data_root, transform=transform_test)

# Split datasets into training and test sets
batch_size = 64
train_size = int(0.8 * len(train_dataset))
test_size = len(train_dataset) - train_size
train_dataset, test_dataset = random_split(train_dataset, [train_size, test_size])

# Load data
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


# Baseline Model

In [21]:
# MLP Model for baseline comparison
class MLPModel(nn.Module):
    def __init__(self, input_size=128*128*3, hidden_size=100, num_classes=3):
        super(MLPModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten the image
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Initialize MLP model, criterion, and optimizer
mlp_model = MLPModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mlp_model.parameters(), lr=0.001)


In [22]:
def train_mlp_model(data_loader):
    mlp_model.train()
    num_epochs = 10
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in data_loader:
            images = images.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = mlp_model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(data_loader):.4f}")

# Train MLP model
train_mlp_model(train_loader)


Epoch [1/10], Loss: 3.1625
Epoch [2/10], Loss: 1.2496
Epoch [3/10], Loss: 1.2257
Epoch [4/10], Loss: 1.1320
Epoch [5/10], Loss: 1.2585
Epoch [6/10], Loss: 1.0754
Epoch [7/10], Loss: 1.0385
Epoch [8/10], Loss: 1.0268
Epoch [9/10], Loss: 1.0256
Epoch [10/10], Loss: 0.9970


In [23]:
def evaluate_mlp_model(model, train_loader, test_loader):
    model.eval()
    
    def evaluate(data_loader):
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in data_loader:
                images = images.to(device)
                labels = labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        accuracy = 100 * correct / total
        return accuracy
    
    train_accuracy = evaluate(train_loader)
    test_accuracy = evaluate(test_loader)
    
    print(f'MLP Model Train Accuracy: {train_accuracy:.2f}%')
    print(f'MLP Model Test Accuracy: {test_accuracy:.2f}%')
    
    return train_accuracy, test_accuracy

# Evaluate MLP Model
evaluate_mlp_model(mlp_model, train_loader, test_loader)


MLP Model Train Accuracy: 53.65%
MLP Model Test Accuracy: 51.39%


(53.651059085841695, 51.39353400222966)

# Convolutional Neural Network

In [24]:


# CNN Model with dynamic flattened layer size
class CNN(nn.Module):
    def __init__(self, layers):
        super(CNN, self).__init__()
        self.model = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.model(x)

# Build CNN model with variable convolutional layers
def build_cnn(num_conv_layers):
    layers = []
    in_channels = 3
    for i in range(num_conv_layers):
        layers.extend([
            nn.Conv2d(in_channels, 6 * (2 ** i), kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        ])
        in_channels = 6 * (2 ** i)

    # Calculate the flattened dimension after convolutional layers
    dummy_input = torch.randn(1, 3, 128, 128).to(device)  # Match input image size here
    with torch.no_grad():
        # Move layers to device (GPU) for dummy calculation
        dummy_output = nn.Sequential(*layers).to(device)(dummy_input)
    flattened_size = dummy_output.view(-1).shape[0]

    # Add fully connected layers
    layers.extend([
        nn.Flatten(),
        nn.Linear(flattened_size, 128),
        nn.ReLU(),
        nn.Linear(128, 3)  # Adjust for number of classes
    ])
    
    return CNN(layers).to(device)



# Define train_and_evaluate function
def train_and_evaluate(model, train_loader, test_loader, criterion, optimizer, epochs=5):
    model.to(device)
    for epoch in range(epochs):
        model.train()
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
    
    # Evaluation
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Accuracy: {accuracy:.2f}%")
    return accuracy

# Initialize models with 1, 2, and 3 convolutional layers
models = {
    "1 Conv Layer": build_cnn(1),
    "2 Conv Layers": build_cnn(2),
    "3 Conv Layers": build_cnn(3)
}

In [25]:
# %%
def train_and_evaluate(model, train_loader, test_loader, criterion, optimizer, epochs=5):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss / len(train_loader):.4f}")
    
    # Evaluation on test set
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    print(f'Accuracy on test set: {accuracy:.2f}%')
    return accuracy

# Dictionary to store model accuracies
accuracies = {}


In [26]:

# Define cross-validation function
def cross_validate_model(model, train_dataset, folds=3):
    kf = KFold(n_splits=folds, shuffle=True)
    accuracies = []
    for fold, (train_idx, val_idx) in enumerate(kf.split(train_dataset)):
        train_subset = Subset(train_dataset, train_idx)
        val_subset = Subset(train_dataset, val_idx)
        train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
        valloader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)
        
        # Optimizer and criterion
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        criterion = nn.CrossEntropyLoss()
        
        # Train and evaluate
        accuracy = train_and_evaluate(model, train_loader, valloader, criterion, optimizer)
        accuracies.append(accuracy)
    
    avg_accuracy = np.mean(accuracies)
    print(f"Average cross-validated accuracy: {avg_accuracy:.2f}%")
    return avg_accuracy

# Test different optimizers
def test_optimizers(model, optimizers, train_loader, test_loader, epochs=5):
    results = {}
    for opt_name, opt_fn in optimizers.items():
        model_copy = build_cnn(2)  # Build a new model instance
        optimizer = opt_fn(model_copy.parameters())
        criterion = nn.CrossEntropyLoss()
        print(f"\nTesting optimizer: {opt_name}")
        accuracy = train_and_evaluate(model_copy, train_loader, test_loader, criterion, optimizer, epochs)
        results[opt_name] = accuracy
    return results

# Optimizer configurations
optimizers = {
    "SGD": lambda params: optim.SGD(params, lr=0.001, momentum=0.9),
    "Adam": lambda params: optim.Adam(params, lr=0.001),
    "RMSProp": lambda params: optim.RMSprop(params, lr=0.001)
}


In [ ]:
# Initialize accuracies dictionary to store model accuracies
accuracies = {}

# Training and recording each model's accuracy
for model_name, model in models.items():
    print(f"\nTraining model: {model_name}")
    accuracy = train_and_evaluate(model, train_loader, test_loader, nn.CrossEntropyLoss(), optim.Adam(model.parameters()))
    accuracies[model_name] = accuracy

# Cross-validation
for model_name, model in models.items():
    print(f"\nCross-validating model: {model_name}")
    avg_accuracy = cross_validate_model(model, train_dataset)
    accuracies[f"{model_name} (CV)"] = avg_accuracy

# Test different optimizers
optimizer_results = test_optimizers(build_cnn(2), optimizers, train_loader, test_loader)
accuracies.update(optimizer_results)

# Display collected accuracies
print("\nModel Accuracies:")
for model, acc in accuracies.items():
    print(f"{model}: {acc:.2f}%")


Training model: 1 Conv Layer
Epoch [1/5], Loss: 1.1422
Epoch [2/5], Loss: 1.0078
Epoch [3/5], Loss: 0.9949
Epoch [4/5], Loss: 0.9791
Epoch [5/5], Loss: 0.9434
Accuracy on test set: 53.29%

Training model: 2 Conv Layers
Epoch [1/5], Loss: 1.1228


In [ ]:
# %%
# Plotting results
plt.figure(figsize=(12, 6))
plt.barh(list(accuracies.keys()), list(accuracies.values()))
plt.xlabel("Accuracy (%)")
plt.ylabel("Model Configuration")
plt.title("Model Comparison with Varying Parameters and Optimizers")
plt.show()


In [ ]:
# %%
model_resnet = resnet50(weights="IMAGENET1K_V2")
for param in model_resnet.parameters():
    param.requires_grad = False  # Freeze all layers

# Modify the last layer to output 3 classes
num_features = model_resnet.fc.in_features
model_resnet.fc = nn.Linear(num_features, 3)
model_resnet = model_resnet.to(device)

# Define optimizer and loss function
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_resnet.parameters(), lr=0.001)

# Train and evaluate transfer learning model
print("\nTraining Transfer Learning Model (ResNet-50):")
transfer_learning_accuracy = train_and_evaluate(model_resnet, train_loader, test_loader, criterion, optimizer, epochs=10)
accuracies["Transfer Learning (ResNet-50)"] = transfer_learning_accuracy


In [ ]:
import matplotlib.pyplot as plt

# Update train_and_evaluate to track both training and test accuracy per epoch
def train_and_evaluate(model, train_loader, test_loader, criterion, optimizer, epochs=10):
    model.to(device)
    train_accuracies = []  # Store training accuracy for each epoch
    test_accuracies = []   # Store test accuracy for each epoch
    
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct_train = 0
        total_train = 0
        
        # Training loop
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            
            # Calculate training accuracy
            _, predicted = torch.max(outputs, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()
        
        # Calculate and store training accuracy
        train_accuracy = 100 * correct_train / total_train
        train_accuracies.append(train_accuracy)
        
        # Calculate test accuracy
        model.eval()
        correct_test = 0
        total_test = 0
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                total_test += labels.size(0)
                correct_test += (predicted == labels).sum().item()
        
        # Calculate and store test accuracy
        test_accuracy = 100 * correct_test / total_test
        test_accuracies.append(test_accuracy)
        
        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {running_loss / len(train_loader):.4f}, "
              f"Train Accuracy: {train_accuracy:.2f}%, Test Accuracy: {test_accuracy:.2f}%")
    
    return train_accuracies, test_accuracies

# Initialize and train the model while tracking accuracies
print("\nTraining Transfer Learning Model (ResNet-50):")
train_accuracies, test_accuracies = train_and_evaluate(model_resnet, train_loader, test_loader, criterion, optimizer, epochs=10)

# Plotting train and test accuracy over epochs
plt.figure(figsize=(10, 6))
plt.plot(range(1, 11), train_accuracies, marker='o', label='Train Accuracy')
plt.plot(range(1, 11), test_accuracies, marker='o', label='Test Accuracy')
plt.xlabel("Epoch")
plt.ylabel("Accuracy (%)")
plt.title("Train and Test Accuracy per Epoch for Transfer Learning (ResNet-50)")
plt.legend()
plt.show()


In [ ]:
def train_and_evaluate(model, train_loader, test_loader, criterion, optimizer, epochs=10):
    model.to(device)
    train_accuracies = []  # Store training accuracy for each epoch
    test_accuracies = []   # Store test accuracy for each epoch
    
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct_train = 0
        total_train = 0
        
        # Training loop
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            
            # Calculate training accuracy
            _, predicted = torch.max(outputs, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()
        
        # Calculate and store training accuracy
        train_accuracy = 100 * correct_train / total_train
        train_accuracies.append(train_accuracy)
        
        # Calculate test accuracy
        model.eval()
        correct_test = 0
        total_test = 0
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                total_test += labels.size(0)
                correct_test += (predicted == labels).sum().item()
        
        # Calculate and store test accuracy
        test_accuracy = 100 * correct_test / total_test
        test_accuracies.append(test_accuracy)
        
        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {running_loss / len(train_loader):.4f}, "
              f"Train Accuracy: {train_accuracy:.2f}%, Test Accuracy: {test_accuracy:.2f}%")
    
    return train_accuracies, test_accuracies

# Initialize and train the model while tracking accuracies
print("\nTraining Transfer Learning Model (ResNet-50):")
train_accuracies, test_accuracies = train_and_evaluate(model_resnet, train_loader, test_loader, criterion, optimizer, epochs=10)

# Plotting train and test accuracy over epochs
plt.figure(figsize=(10, 6))
plt.plot(range(1, 11), train_accuracies, marker='o', label='Train Accuracy')
plt.plot(range(1, 11), test_accuracies, marker='o', label='Test Accuracy')
plt.xlabel("Epoch")
plt.ylabel("Accuracy (%)")
plt.title("Train and Test Accuracy per Epoch for Transfer Learning (ResNet-50)")
plt.legend()
plt.show()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
import time

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')

# Load dataset and split into training and testing sets
transform = transforms.Compose([
    transforms.Resize((100, 100)),
    transforms.ToTensor(),
])

data_dir = "train_data"
dataset = datasets.ImageFolder(root=data_dir, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Data loaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

# Load ResNet-50 model with pretrained weights and modify the final layer
CNN = models.resnet50(weights="IMAGENET1K_V2")  # Use pre-trained weights
CNN.fc = nn.Linear(CNN.fc.in_features, 3)  # Adjust final layer for 3 classes
CNN = CNN.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(CNN.parameters(), lr=0.0001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)  # Reduce LR every 7 epochs

# Training parameters
num_epochs = 10
train_losses, test_losses = [], []
train_accuracies, test_accuracies = [], []

# Training and evaluation function
def train_and_evaluate(model, criterion, optimizer, scheduler, num_epochs=10):
    start_time = time.time()

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss, correct, total = 0.0, 0, 0
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
        
        train_losses.append(train_loss / total)
        train_accuracies.append(correct / total)

        # Testing phase
        model.eval()
        test_loss, correct, total = 0.0, 0, 0

        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                
                test_loss += loss.item() * images.size(0)
                _, predicted = torch.max(outputs, 1)
                correct += (predicted == labels).sum().item()
                total += labels.size(0)
        
        test_losses.append(test_loss / total)
        test_accuracies.append(correct / total)

        scheduler.step()  # Update learning rate

        elapsed_time = time.time() - start_time
        print(f'Epoch [{epoch+1}/{num_epochs}], '
              f'Train Loss: {train_losses[-1]:.4f}, Train Accuracy: {train_accuracies[-1]:.4f}, '
              f'Test Loss: {test_losses[-1]:.4f}, Test Accuracy: {test_accuracies[-1]:.4f}, '
              f'Time: {elapsed_time:.2f}s')

# Train and evaluate
train_and_evaluate(CNN, criterion, optimizer, scheduler, num_epochs)

# Plotting
epochs = range(1, num_epochs + 1)

plt.figure(figsize=(12, 5))

# Loss plot
plt.subplot(1, 2, 1)
plt.plot(epochs, train_losses, label='Training Loss', marker='o')
plt.plot(epochs, test_losses, label='Testing Loss', marker='x')
plt.title('Loss vs Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.xticks(epochs)
plt.grid()
plt.legend()

# Accuracy plot
plt.subplot(1, 2, 2)
plt.plot(epochs, train_accuracies, label='Training Accuracy', marker='o')
plt.plot(epochs, test_accuracies, label='Testing Accuracy', marker='x')
plt.title('Accuracy vs Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.xticks(epochs)
plt.grid()
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
from tqdm import tqdm  # Add this line at the top

# Training and evaluation function with tqdm progress bar
def train_and_evaluate(model, criterion, optimizer, scheduler, num_epochs=10):
    start_time = time.time()

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss, correct, total = 0.0, 0, 0
        
        # Wrap train_loader with tqdm for progress display
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training", leave=False):
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
        
        train_losses.append(train_loss / total)
        train_accuracies.append(correct / total)

        # Testing phase
        model.eval()
        test_loss, correct, total = 0.0, 0, 0

        # Wrap test_loader with tqdm for progress display
        with torch.no_grad():
            for images, labels in tqdm(test_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Testing", leave=False):
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                
                test_loss += loss.item() * images.size(0)
                _, predicted = torch.max(outputs, 1)
                correct += (predicted == labels).sum().item()
                total += labels.size(0)
        
        test_losses.append(test_loss / total)
        test_accuracies.append(correct / total)

        scheduler.step()  # Update learning rate

        elapsed_time = time.time() - start_time
        print(f'Epoch [{epoch+1}/{num_epochs}], '
              f'Train Loss: {train_losses[-1]:.4f}, Train Accuracy: {train_accuracies[-1]:.4f}, '
              f'Test Loss: {test_losses[-1]:.4f}, Test Accuracy: {test_accuracies[-1]:.4f}, '
              f'Time: {elapsed_time:.2f}s')

# Train and evaluate with tqdm progress bar
train_and_evaluate(model_resnet, criterion, optimizer, scheduler, num_epochs)